TO DO

general implementation
    create a graph or diagram to tell whats hapenning
    mark all the tasks/steps complete, incomplete, in progress, problems, to do, research etc.
    try on a toy problem

find new datasets
    for different datasets different preprocessing techniques should be applied
    RCV1-V2
decide on splitting ratio 20 60 20 

try different similarity measures 
    reference paper
    cosine
    euclidean

implementation steps

1. reading data and preprocessing
2. vectorization
    2.1 embeddings
    2.2 dimensionality reduction? (is similarity more accurate when dim. red. done) research
3. initial classifier to show results
4. calculate imbalance ratio and find the ratio of newly labeled data
5. oversample dataset using unlabeled set
    5.1 find the proper similarity function (eclidean, cosine etc.)
        Measurement of Text Similarity: A Survey: a very detailed survey of similarity functions that are used for text data
        https://www.kdnuggets.com/2019/01/comparison-text-distance-metrics.html
        cosine similarity
        minkowski family (euclidean, manhattan)
        hamming distance
        Jaccard index
        Sorensen-dice index
        KL divergence
        Jensen–Shannon divergence with LDA
        Wasserstein distance
        SMTP 
        word mover’s distance
    5.2 define a threshold or mechanism to add data for multilabeled set
6. train a final classifier to compare results

In [ ]:
# import pandas as pd
import numpy as np
import preprocess
from sklearn.svm import LinearSVC
from sklearn.utils.class_weight import compute_class_weight
from sklearn.multiclass import OneVsRestClassifier

In [2]:
def read_data(path):
    df = pd.read_csv(path)
    df['text'] = df['text'].apply(preprocess.preprocess_text)
    return df

In [3]:
def vectorize_data(text, model_name='stsb-roberta-large'):
    
    from sentence_transformers import util
    from sentence_transformers import SentenceTransformer
    import torch
    
    model = SentenceTransformer(model_name)
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    
    vectors = model.encode(text, convert_to_tensor=False, device=device)
    
    return vectors

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
def classifier(X_train, y_train, X_test, y_test):
    
    def calculating_class_weights(y_true):
        
        number_dim = np.shape(y_true)[1]
        weights = []
        for i in range(number_dim):
            at = compute_class_weight('balanced', [0.,1.], y_true[:, i])
            weights.append(dict(zip([0,1], at)))
            # weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])))
        return weights

    # class_weights = calculating_class_weights(y_train.values)
    
    # Linear SVM
    linearSvm = OneVsRestClassifier(LogisticRegression(class_weight='balanced'), n_jobs=-1)
    print(X_train)
    print(y_train)
    linearSvm.fit(X_train.values, y_train)
    linearSvm_preds = linearSvm.predict(X_test)
    print_losses(y_test, linearSvm_preds, 'Linear SVM Classifier')
    
    print("\033[1m" + 'LinearSVM results: ' + "\033[0m")
    print('-'*30)
    hamLoss = hamming_loss(y_test.values, linearSvm_preds)
    print('hamLoss: {:.2f}'.format(hamLoss))
    acc_score = accuracy_score(y_test.values, linearSvm_preds)
    print('Exact Match Ratio: {:.2f}'.format(acc_score))
    print('-'*30)
    print("\033[1m" + 'Classification Report' + "\033[0m")
    print(classification_report(y_test.values, linearSvm_preds, target_names=list(y_test.columns)))

In [6]:
def find_balancing_ratio(y):
    balancing_ratio = 1.0
    return balancing_ratio

In [7]:
def oversample_dataset(labeled_data, unlabeled_data):
    return labeled_data, unlabeled_data

## ## 1. reading and preprocessing data

In [8]:
majority_path = r'C:\Users\IsmailKaraman\workspace\data\privacy_policy_data\OPP-115_v2\majority.csv'
df = pd.read_csv(majority_path)
df.head()

,text,Data Retention,Data Security,Do Not Track,First Party Collection/Use,International and Specific Audiences,Introductory/Generic,Policy Change,Practice not covered,Privacy contact information,Third Party Sharing/Collection,"User Access, Edit and Deletion",User Choice/Control
0,"""""Contact Us"""" Link If you contact us through...",0,0,0,1,0,0,0,0,0,0,0,0
1,(b) Information automatically collected There...,0,0,0,1,0,0,0,0,0,0,0,0
2,(ii) You have entered a contest or sweepstake...,0,0,0,0,0,0,0,0,0,1,0,0
3,*Web Beacons: Military Web pages and the Web ...,0,0,0,1,0,0,0,0,0,1,0,0
4,1. INFORMATION WE COLLECT We collect the foll...,0,0,0,0,0,1,0,0,0,0,0,0


In [9]:
col_names = ['Data Retention', 'Data Security', 'Do Not Track', 'First Party Collection/Use', 
             'International and Specific Audiences',       'Introductory/Generic', 'Policy Change', 
             'Practice not covered', 'Privacy contact information', 'Third Party Sharing/Collection',
             'User Access, Edit and Deletion', 'User Choice/Control']
sub_col_names = ['Data Security', 'User Access, Edit and Deletion', 'Policy Change']

In [10]:
toy_df = df[(df[col_names].sum(axis=1)==df[sub_col_names].sum(axis=1))].sample(100)

In [11]:
toy_df.sum()

text                                     data security reddit will make every effort t...
Data Retention                                                                          0
Data Security                                                                          53
Do Not Track                                                                            0
First Party Collection/Use                                                              0
International and Specific Audiences                                                    0
Introductory/Generic                                                                    0
Policy Change                                                                          29
Practice not covered                                                                    0
Privacy contact information                                                             0
Third Party Sharing/Collection                                                          0
User Acces

In [12]:
X = toy_df['text']
y = toy_df[sub_col_names]

## 2. vectorization

X_num = X.apply(vectorize_data)
import pickle
with open('X_num.p', 'wb') as f:
    pickle.dump(X_num, f)

In [24]:
import pickle
with open('X_num.p', 'rb') as f:
    X_num = pickle.load(f)

In [15]:
X_labeled.shape, X_unlabeled.shape, X_test.size

((20,), (60,), 20)

In [16]:
y_labeled.sum()

Data Security                     10
User Access, Edit and Deletion     4
Policy Change                      6
dtype: int64

## 3. initial classifier to show results

classifier(X_labeled.values, y_labeled, X_test, y_test)

## 4. calculate imbalance ratio and find the ratio of newly labeled data

In [ ]:
def calculate_imb_ratio(y):

    class_ratios = (y.sum() / y.shape[0]).values
    return class_ratios

In [42]:
def cal_balancing_num_instance_binary(n_samples, n_total_samples, balance_ratio=0.5):
    
    if n_samples/n_total_samples > balance_ratio:
        print("Be careful! Given balancing ratio is lower than the class' imbalance ratio")
        
    return int((n_total_samples*balance_ratio - n_samples)*2)

array([1.2195122 , 5.55555556])

In [40]:
def cal_balancing_num_instance_multiclass(y, balance_ratio):
    
    oversampling_counts = {}
    n_samples = y.shape[0]
    n_classes = y.shape[1]
    
    for col in y.columns:
        oversampling_counts[col] = cal_balancing_num_instance_binary(y[col].sum(), n_samples, balance_ratio)
    
    return oversampling_counts

array([82, 18], dtype=int64)

In [38]:
def oversample_dataset(num_of_new_instances, X_labeled, y_labeled, X_unlabeled, y_unlabeled):
    
    for col_name, num_instance in num_of_new_instances.items():
        new_instances = find_new_instances(col_name, num_instance)
    
    add new instances to X_labeled (and y_labeled for test case)
    remove new instances from X_unlabeled (and y_unlabeled for test case)
    
    return 

18

# main

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=0, stratify=y)
X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(X_train, y_train, test_size=0.75, stratify=y_train, random_state=2)

classifier(X_labeled.values, y_labeled, X_test, y_test)

num_of_new_instances = cal_balancing_num_instance_multiclass(y, 0.5)

oversample_dataset(num_of_new_instances, X_labeled, y_labeled, X_unlabeled, y_unlabeled)

    
    
    
    
    

In [ ]:
from nltk.corpus import words
all_words = words.words()

In [ ]:
words_5len = [word.lower() for word in all_words if len(word)==5]

In [ ]:
for word in words_5len:
    if 'h' in word and 'k' in word and 'n' in word:
        print(word)

In [ ]:
for word in words_5len:
    if word.startswith('se') and 'r' in word:
        print(word)

In [ ]:
letter_0 = ''
letter_1 = ''
letter_2 = 'a'
letter_3 = ''
letter_4 = ''

exist_letters = 'acs'
banned_letters = 'trdefou'

In [ ]:
filtered = [word for word in words_5len for e in exist_letters if e in word]
filtered

In [ ]:
filtered = [word for word in filtered for b in banned_letters if b in word]
filtered = [word for word in filtered if letter_0 and word[0]==letter_0]
filtered = [word for word in filtered if letter_1 and word[0]==letter_1]
filtered = [word for word in filtered if letter_2 and word[0]==letter_2]
filtered = [word for word in filtered if letter_3 and word[0]==letter_3]
filtered = [word for word in filtered if letter_4 and word[0]==letter_4]
filtered

In [ ]:
if letter_2:
    print('a')

In [ ]:
for word in words_5len:
    if 'o' in word and 'u' in word and 'a' not in word and 'i' not in word and 'd' not in word:
        print(word)